지오코딩 + 카카오맵

In [8]:
import requests
import pandas as pd
from tqdm import tqdm
# :작은_파란색_다이아몬드: 카카오 REST API 키 입력 (본인의 키로 변경)
KAKAO_API_KEY = "9604fec7266f3ea34c864d4c94f62e5c"
# :작은_파란색_다이아몬드: 카카오 지오코딩 API 요청 함수
def get_coordinates(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["documents"]:
            x = data["documents"][0]["x"]  # 경도 (longitude)
            y = data["documents"][0]["y"]  # 위도 (latitude)
            return x, y
    return None, None  # 주소가 없을 경우
# :작은_파란색_다이아몬드: CSV 파일 불러오기 (지번 주소가 포함된 CSV 파일)
df = pd.read_csv("2015_아파트(매매)_실거래가.csv", encoding="utf-8")  # 인코딩 확인 필요!
coordinates = [get_coordinates(addr) for addr in tqdm(df["주소"], desc="주소 좌표 변환 중")]
# :작은_파란색_다이아몬드: 주소를 위도, 경도로 변환하여 새로운 컬럼 추가
df["경도"], df["위도"] = zip(*df["주소"].apply(get_coordinates))
# :작은_파란색_다이아몬드: 변환된 데이터 저장
df.to_csv("2015_아파트(매매)_실거래가_좌표변환.csv", encoding="utf-8", index=False)
print("좌표 변환 완료! :짠:")

주소 좌표 변환 중: 100%|██████████| 8408/8408 [10:08<00:00, 13.83it/s]


좌표 변환 완료! :짠:


In [3]:
import pandas as pd 

df = pd.read_csv('2010_아파트(매매)_실거래가_좌표변환.csv')
df.head(10)

,NO,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,주소,경도,위도
0,1,서울특별시 강서구 내발산동,753,753,0,마곡수명산파크4단지,59.84,201012,31,"36,800",8,2008,수명로1길 16,서울특별시 강서구 내발산동 753,126.823351,37.551235
1,2,서울특별시 강서구 화곡동,1159,1159,0,"우장산아이파크,이편한세상",84.97,201012,31,"60,500",20,2008,강서로 266,서울특별시 강서구 화곡동 1159,126.838766,37.549355
2,3,서울특별시 강서구 화곡동,1159,1159,0,"우장산아이파크,이편한세상",84.97,201012,31,"53,600",2,2008,강서로 266,서울특별시 강서구 화곡동 1159,126.838766,37.549355
3,4,서울특별시 강서구 화곡동,1159,1159,0,"우장산아이파크,이편한세상",59.98,201012,31,"44,000",12,2008,강서로 266,서울특별시 강서구 화곡동 1159,126.838766,37.549355
4,5,서울특별시 강서구 화곡동,1159,1159,0,"우장산아이파크,이편한세상",84.97,201012,31,"60,300",7,2008,강서로 266,서울특별시 강서구 화곡동 1159,126.838766,37.549355
5,6,서울특별시 강서구 등촌동,713,713,0,대동황토방2,80.93,201012,31,"40,000",2,2003,공항대로55길 50,서울특별시 강서구 등촌동 713,126.862402,37.555177
6,7,서울특별시 강서구 등촌동,707,707,0,"등촌7,8단지주공아파트",41.85,201012,31,"20,500",10,1994,공항대로43길 104,서울특별시 강서구 등촌동 707,126.851688,37.560804
7,8,서울특별시 강서구 등촌동,695,695,0,주공5,58.14,201012,31,"31,000",13,1995,공항대로39길 74,서울특별시 강서구 등촌동 695,126.844869,37.560452
8,9,서울특별시 강서구 등촌동,691-3,691,3,부영,80.64,201012,31,"40,300",5,1994,강서로56나길 110,서울특별시 강서구 등촌동 691-3,126.843641,37.563558
9,10,서울특별시 강서구 방화동,862,862,0,한숲마을대림,84.83,201012,31,"33,000",8,1999,방화대로50길 7,서울특별시 강서구 방화동 862,126.817514,37.577249


In [4]:
print(data.columns)

Index(['NO', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '계약년월', '계약일',
       '거래금액(만원)', '층', '건축년도', '도로명', '주소', '경도', '위도'],
      dtype='object')


In [7]:
import pandas as pd
import folium
import branca.colormap as cm

# CSV 파일 읽기
file_name = '2010_아파트(매매)_실거래가_좌표변환.csv'
data = pd.read_csv(file_name)

# '위도', '경도', '전용면적(㎡)' 컬럼 확인 및 결측값 처리
if '위도' in data.columns and '경도' in data.columns:
    # 위도/경도 결측값 제거
    data = data.dropna(subset=['위도', '경도'])

    # 전용면적(㎡) 컬럼 처리
    if '전용면적(㎡)' in data.columns:
        data['전용면적(㎡)'] = pd.to_numeric(data['전용면적(㎡)'], errors='coerce')  # 숫자형 변환
        data = data.dropna(subset=['전용면적(㎡)'])  # 결측값 제거
    else:
        print("'전용면적(㎡)' 컬럼이 없습니다.")
        exit()

    # 전용면적(㎡)에 따라 색상 매핑
    colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], 
                                 vmin=data['전용면적(㎡)'].min(), 
                                 vmax=data['전용면적(㎡)'].max())

    # 지도 생성 (서울 중심 좌표로 설정)
    seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

    # 데이터의 각 행에 대해 마커 추가
    for idx, row in data.iterrows():
        try:
            latitude = row['위도']
            longitude = row['경도']
            area = row['전용면적(㎡)']

            # 지도에 마커 추가 (색상 적용)
            folium.CircleMarker(
                location=[latitude, longitude],
                radius=7,
                color=colormap(area),
                fill=True,
                fill_color=colormap(area),
                fill_opacity=0.7,
                popup=f"전용면적: {area}㎡"
            ).add_to(seoul_map)
        except Exception as e:
            print(f"마커 추가 중 오류 발생 (행 {idx}): {e}")

    # 색상 범례 추가
    colormap.caption = '전용면적(㎡)'
    seoul_map.add_child(colormap)

    # 결과 지도 저장
    output_file = '아파트_실거래가_지도_색상.html'
    seoul_map.save(output_file)
    print(f"지도 저장 완료: {output_file}")
else:
    print("'위도' 또는 '경도' 컬럼이 데이터에 없습니다.")


지도 저장 완료: 아파트_실거래가_지도_색상.html
